In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

[link text](https://)
# Using the Vertex AI Gemini API to explain BigQuery ML Clustering

link: https://github.com/GoogleCloudPlatform/genai-for-marketing/blob/main/notebooks/explaining_cluster_analysis_with_bqml_and_vertexai_gemini.ipynb


| | |
|----------|-------------|
| Author(s)   | Hussain Chinoy (ghchinoy@) |



## Overview

This example demostrates how to use the Vertex AI Gemini API to explain BigQuery ML clustering.

[BigQuery ML (BQML)](https://cloud.google.com/bigquery/docs/bqml-introduction) is a powerful way for analysts to utilize various ML techniques within a familiar relational data store - BigQuery.

Typically, the end goal is to explain the results of analysis - this is where the Vertex AI Gemini LLM API can come in handy.

### About Clustering
Custering, also called segmentation, is a common exploratory technique to discover different groupings of behavior types based on a collection of known attributes within data, in this case, consumer purchasing patterns. We can imagine people following different purchasing patterns based upon our lived experiences, but using data and BQML in particular provides and uncovers segmentation in interesting and reproducible ways.

Segmentation is used to identify naturally occurring groupings of customers who have similar characteristics, such as purchasing habits based upon stores visited, items purchased, or price paid for those items. Or, in this example, to design a more meaningful marketing campaign that appeals to a specific group.

The aim of customer segmentation is to split customers by attributes and one common method of segmentation is the “RFM model”
* Recency  - How recently did a customer purchase or what’s the avg gap between purchases?
* Frequency - How often did they purchase?
* Monetary value - How much did they spend?

Some other variations are
* RFE - Recency, Frequency, Engagement - visit duration, pages per visit, etc.
* RFD - Recency, Frequency, Duration - length of time spent browsing on a page (advertising use cases)

Using a public data source via Analytics Hub, we'll the publicly available synthetic eCommerce data set to our BigQuery instance, create a cluster, and then explain it with Gemini.


Learn more about [BigQuery ML](https://cloud.google.com/bigquery/docs/bqml-introduction) and the [Vertex AI Gemini API](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview)


### Objective

In this tutorial, you learn how to use BigQuery ML (BQML) to cluster ecommerce users based on spend history and then explain those clusters using the Gemini Foundational Large Multimodal Model, making cluster analysis more accessible to your team!

This tutorial uses the following Google Cloud ML services and resources:

- BigQuery
- BigQuery ML
- Vertex AI Gemini


The steps performed include:

- Take a peek at the data
- Create a K-means cluster with BQML
- Explain that cluster results with Gemini


### Dataset

For this example, we'll use a [BigQuery Public Dataset](https://console.cloud.google.com/marketplace/browse?filter=partner:BigQuery%20Public%20Data) that contains synthetic eCommerce and Digital Marketing data for the ficitious company called TheLook. Here's the link to the dataset [theLook eCommerce](https://console.cloud.google.com/marketplace/product/bigquery-public-data/thelook-ecommerce)

### Costs

This tutorial uses billable components of Google Cloud:

* BigQuery
* Generative AI support on Vertex AI

Learn about [BigQuery compute pricing](https://cloud.google.com/bigquery/pricing#analysis_pricing_models),
[Generative AI support on Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing#generative_ai_models), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.


Let's log in with Google, load the Vertex AI libraries and restart the runtime

## Setup


### Required SDKs

Let's install the required SDKs

In [ ]:
!pip install google-cloud-aiplatform --upgrade --user --quiet
!pip install bigframes --upgrade --quiet

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ Before proceeding, please wait for the kernel to finish restarting ⚠️</b>
</div>

### Authenticate your Google Account

Depending on your environment, you may have to manually authenticate. Please follow instructions below.

For BigQuery notebooks and Vertex AI Workbench, you're already authenticated therefore you can skip these instructions!

#### Authentication (Colab Only)

In [ ]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

## Imports and project variables


Let's define some variables that will be used throughout this notebook.

These are the GCP Project ID `project_id`, the Model name `model_name` which is any name you prefer, and finally the Dataset name `dataset_name`.
The dataset needs to exist in the same Project as `project_id` and you'll need appropriate access to create and delete.

In [1]:
import sys
import pandas as pd
from typing import Union
from google.cloud import bigquery
import bigframes.pandas as bpd

In [ ]:
#@title Setup Project Variables { run: "auto", display-mode: "form" }
project_id = '<your_project_id>' #@param {type:"string"}
dataset_name = "bqml_clustering" #@param {type:"string"}
model_name = "ecommerce_customer_segmentation" #@param {type:"string"}
eval_name = model_name + "_eval"
LOCATION = "us-central1"  # @param {type:"string"}
client = bigquery.Client(project=project_id)

# creates the target dataset
try:
    dataset = bigquery.Dataset(f"{project_id}.{dataset_name}")
    dataset = client.create_dataset_if_not_exists(dataset)
except:
    print(f"Dataset {dataset_name} already exists. Please continue.")



## Create a K-means model to cluster ecommerce data

First let's look at our data quickly before we create the model. This query can be run in BigQuery on its own. Set dates accordingly. Try it out!

In [ ]:
query = """
SELECT
  user_id,
  order_id,
  sale_price,
  created_at as order_created_date
FROM `bigquery-public-data.thelook_ecommerce.order_items`
WHERE created_at BETWEEN CAST('2020-01-01 00:00:00' AS TIMESTAMP)
AND CAST('2025-03-01 00:00:00' AS TIMESTAMP)
"""
#client = bigquery.Client()
#df = client.query(query).to_dataframe()
#df.head()

bq_df = bpd.read_gbq(query)
bq_df.head()

In [ ]:
# prompt: a dataframe with product data

import pandas as pd
query = """
SELECT
    *
  FROM
    `bigquery-public-data`.thelook_ecommerce.products
LIMIT 10
"""
product_df = bpd.read_gbq(query)
product_df.head()

## `CREATE MODEL` using `KMEANS`

Create a query then start the model creation job, using a python loop to wait for the job to complete. Please note, if you've created the model already, there's no need to rerun this step to create the clustering using model on the data.

In [4]:
#@title KMEANS model  { run: "auto", display-mode: "form" }

BEGIN_DATE = "\"2020-01-01\"" #@param {type:"string"}
END_DATE = "\"2025-03-01\"" #@param {type:"string"}

OUR_CURR_DATE = "\"2025-03-12\"" #@param {type:"string"}

query = """
CREATE OR REPLACE MODEL `{0}.{1}`
-- CREATE MODEL IF NOT EXISTS `{0}.{1}`
OPTIONS (
  MODEL_TYPE = "KMEANS",
  NUM_CLUSTERS = HPARAM_RANGE(5,7),
  KMEANS_INIT_METHOD = "KMEANS++",
  STANDARDIZE_FEATURES = TRUE,
  HPARAM_TUNING_ALGORITHM = 'VIZIER_DEFAULT',
  NUM_TRIALS = 10,
  MODEL_REGISTRY = 'VERTEX_AI',
  VERTEX_AI_MODEL_ID = '{0}_{1}')
AS (
SELECT * EXCEPT (user_id)
FROM (
  SELECT user_id,
    DATE_DIFF(DATE({2}), CAST(MAX(order_created_date) as DATE), day) AS days_since_order, -- RECENCY
    COUNT(DISTINCT order_id) AS count_orders, -- FREQUENCY
    AVG(sale_price) AS avg_spend -- MONETARY
  FROM (
    SELECT user_id,
      order_id,
      sale_price,
      created_at as order_created_date
    FROM `bigquery-public-data.thelook_ecommerce.order_items`
    WHERE created_at BETWEEN TIMESTAMP({3})
    AND TIMESTAMP({4})
  )
  GROUP BY user_id
 )
)
""".format(dataset_name, model_name, OUR_CURR_DATE, BEGIN_DATE, END_DATE)


In [5]:
# Wrapper to use BigQuery client to run query/job, return job ID or result as DF
def run_bq_query(sql: str, use_query_cache: bool = False) -> Union[str, pd.DataFrame]:
    """
    Input: SQL query, as a string, to execute in BigQuery
    Returns the query results as a pandas DataFrame, or error, if any
    """

    # Try dry run before executing query to catch any errors
    job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=use_query_cache)
    client.query(sql, job_config=job_config)

    # If dry run succeeds without errors, proceed to run query
    job_config = bigquery.QueryJobConfig(use_query_cache=use_query_cache)
    client_result = client.query(sql, job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running. then get & return data frame
    df = client_result.result().to_arrow().to_pandas()
    print(f"Finished job_id: {job_id}")
    return df

In [ ]:
print(query)

# create the model
run_bq_query(query)

Let's take a look at the model's clustering performance, using these metrics - [Davies Bouldin Index](https://en.wikipedia.org/wiki/Davies%E2%80%93Bouldin_index) and Mean Squared Distance

In [7]:
query = """
SELECT *
FROM ML.EVALUATE(MODEL `{0}.{1}`)
""".format(dataset_name, model_name)

run_bq_query(query)


Finished job_id: 37f529cd-41ea-4742-ad46-681f48b29cf0


,trial_id,davies_bouldin_index,mean_squared_distance
0,1,0.945432,0.774159
1,2,1.051601,0.785036
2,3,0.873953,0.954535


In [8]:
query = """
SELECT *
FROM ML.TRIAL_INFO(MODEL `{0}.{1}`)
""".format(dataset_name, model_name)

run_bq_query(query)

Finished job_id: c12b9426-ed1f-477d-9947-a1cc54cafef9


,trial_id,hyperparameters,hparam_tuning_evaluation_metrics,training_loss,eval_loss,status,error_message,is_optimal
0,1,{'num_clusters': 6},{'davies_bouldin_index': 0.9454321678567678},0.774159,NaN,SUCCEEDED,None,False
1,2,{'num_clusters': 7},{'davies_bouldin_index': 1.0516013343110342},0.785036,NaN,SUCCEEDED,None,False
2,3,{'num_clusters': 5},{'davies_bouldin_index': 0.873952536561688},0.954535,NaN,SUCCEEDED,None,True


Let's analyse the features info

In [9]:
query = """
SELECT *
FROM ML.FEATURE_INFO(MODEL `{0}.{1}`)
""".format(dataset_name, model_name)

run_bq_query(query)

Finished job_id: 980a0cc3-5ba2-4228-a816-1e87370570d1


,input,min,max,mean,median,stddev,category_count,null_count,dimension,entropy
0,days_since_order,12.00,1897.0,482.392547,335.000000,439.697921,NaN,0,NaN,4.199886
1,count_orders,1.00,4.0,1.545177,1.000000,0.849137,NaN,0,NaN,0.973213
2,avg_spend,0.49,999.0,59.557745,46.989999,52.875574,NaN,0,NaN,2.606042


Now let's get the cluster (centroid) information

In [10]:
query = """
SELECT
  CONCAT('cluster ', CAST(centroid_id as STRING)) as centroid,
  avg_spend as average_spend,
  count_orders as count_of_orders,
  days_since_order
FROM (
  SELECT centroid_id, feature, ROUND(numerical_value, 2) as value
  FROM ML.CENTROIDS(MODEL `{0}.{1}`) centroids
  JOIN ML.TRIAL_INFO(MODEL `{0}.{1}`) info
  ON centroids.trial_id = info.trial_id
  WHERE info.is_optimal = TRUE
)
PIVOT (
  SUM(value)
  FOR feature IN ('avg_spend',  'count_orders', 'days_since_order')
)
ORDER BY centroid_id
""".format(dataset_name, model_name)
run_bq_query(query)

Finished job_id: 26b72b60-1c8f-474d-bd3c-7dafc69bf691


,centroid,average_spend,count_of_orders,days_since_order
0,cluster 1,45.62,1.33,283.28
1,cluster 2,48.91,1.16,1170.13
2,cluster 3,58.40,3.48,252.51
3,cluster 4,475.56,1.17,579.97
4,cluster 5,162.58,1.26,434.30


Whew! That's a lot of metrics and cluster info. How about we explain this to our colleagues using the magic of LLMs.

In [11]:
df = client.query(query).to_dataframe()
df.to_string(header=False, index=False)

cluster_info = []
for i, row in df.iterrows():
  cluster_info.append("{0}, average spend ${2}, count of orders per person {1}, days since last order {3}"
    .format(row["centroid"], row["count_of_orders"], row["average_spend"], row["days_since_order"]) )

print(str.join("\n", cluster_info))

cluster 1, average spend $45.62, count of orders per person 1.33, days since last order 283.28
cluster 2, average spend $48.91, count of orders per person 1.16, days since last order 1170.13
cluster 3, average spend $58.4, count of orders per person 3.48, days since last order 252.51
cluster 4, average spend $475.56, count of orders per person 1.17, days since last order 579.97
cluster 5, average spend $162.58, count of orders per person 1.26, days since last order 434.3


## Explain with Vertex AI Gemini API

Initialize the Vertex AI python SDK

In [12]:
import vertexai
vertexai.init(project=project_id, location=LOCATION)

Generate a text prediction

In [13]:
from vertexai.generative_models import GenerationConfig, GenerativeModel
import typing_extensions as typing
import json
from pandas import json_normalize



model = GenerativeModel("gemini-2.0-flash-001")

clusters = str.join("\n", cluster_info)

prompt = f"""
You're a creative brand strategist, given the following clusters, come up with creative brand persona, a catchy title, and next marketing action, explained step by step.

Clusters:
{clusters}

For each Cluster:
* Title:
* Business oriented description of the custer :
* Persona:
* Next Marketing Step:
"""


generation_config = GenerationConfig(
    temperature=0.55,
    top_p=0.8,
    top_k=40,
    candidate_count=1,
    max_output_tokens=8192,
)

responses = model.generate_content(
    prompt,
    generation_config=generation_config,
    #stream=True,
)
output= responses.candidates[0].content.parts[0].text
print(output)

#for response in responses:
#        print(response.text, end="")


Okay, here's a breakdown of each cluster with a catchy title, business-oriented description, brand persona, and a suggested next marketing action.

**Cluster 1: "The Occasional Shopper"**

*   **Title:** The Occasional Shopper
*   **Business-Oriented Description:** This cluster represents customers with a low average spend, infrequent purchase frequency, and a long recency period (almost a year since their last purchase). They are likely price-sensitive and not highly engaged with the brand.
*   **Persona:**
    *   **Name:** Sarah
    *   **Age:** 32
    *   **Occupation:** Teacher
    *   **Lifestyle:** Budget-conscious, values quality but shops around for the best deals. Purchases are often driven by specific needs or events.
    *   **Motivations:** Value, convenience, and a clear understanding of the product benefits.
    *   **Pain Points:** Overwhelmed by choices, hesitant to spend money on non-essentials, forgets about the brand.
*   **Next Marketing Step:**
    1.  **Re-engage

Voila!  🎉 We've now used k-means clustering to create groups of spenders and explain their profiles.

> Add blockquote



### Predict customer's clusters


In [14]:
query="""
WITH test_customers AS (
  SELECT "alice" AS user_id, 200 AS days_since_order, 1 AS count_orders, 100 AS avg_spend
  UNION ALL
  SELECT "david",120,2,300
  UNION ALL
  SELECT "sarah",365,3,250
)
SELECT *
FROM
ML.PREDICT(
  MODEL `{0}.{1}`,
  (
    SELECT *
    FROM
      test_customers
  ));
""".format(dataset_name, model_name)

In [15]:
print(query)

run_bq_query(query)


WITH test_customers AS (
  SELECT "alice" AS user_id, 200 AS days_since_order, 1 AS count_orders, 100 AS avg_spend
  UNION ALL
  SELECT "david",120,2,300
  UNION ALL
  SELECT "sarah",365,3,250
)
SELECT *
FROM
ML.PREDICT(
  MODEL `bqml_clustering.ecommerce_customer_segmentation`,
  (
    SELECT *
    FROM
      test_customers
  ));

Finished job_id: d50dc9b6-8ebf-4ef4-b3cb-980773aaf0f0


,trial_id,CENTROID_ID,NEAREST_CENTROIDS_DISTANCE,user_id,days_since_order,count_orders,avg_spend
0,3,5,"[{'CENTROID_ID': 5, 'DISTANCE': 2.635600573683...",sarah,365,3,250
1,3,5,"[{'CENTROID_ID': 5, 'DISTANCE': 2.831957161755...",david,120,2,300
2,3,1,"[{'CENTROID_ID': 1, 'DISTANCE': 1.116054500565...",alice,200,1,100


In [16]:
print(str.join('\n',cluster_info))

cluster 1, average spend $45.62, count of orders per person 1.33, days since last order 283.28
cluster 2, average spend $48.91, count of orders per person 1.16, days since last order 1170.13
cluster 3, average spend $58.4, count of orders per person 3.48, days since last order 252.51
cluster 4, average spend $475.56, count of orders per person 1.17, days since last order 579.97
cluster 5, average spend $162.58, count of orders per person 1.26, days since last order 434.3


Sometimes, though, you want a little bit [extra](https://cloud.google.com/blog/transform/prompt-debunking-five-generative-ai-misconceptions).

In [18]:
from vertexai.generative_models import GenerationConfig, GenerativeModel

model = GenerativeModel("gemini-2.0-flash-lite-preview-02-05")

cluster_info = str.join('\n', cluster_info)

category_list = "sport retail item"

prompt = f"""
Pretend you're a creative strategist, analyse the following clusters and come up with \
creative brand persona for each that includes the detail of which which product item belonging to a category among this list of categories : [ {category_list} ] is \
likely to be their favorite, a summary of how this relates to their purchasing behavior, \
and a witty e-mail headline for marketing campaign targeted to their group.

Clusters:
{cluster_info}
"""

generation_config = GenerationConfig(
    temperature=0.55,
    top_p=0.8,
    top_k=40,
    candidate_count=1,
    max_output_tokens=1024,
)

responses = model.generate_content(
    prompt,
    generation_config=generation_config,
    #stream=True,
)
print(responses.candidates[0].content.parts[0].text)

Okay, let's dive into these customer clusters and craft some compelling brand personas!

**Cluster Analysis & Brand Personas**

Here's a breakdown of each cluster, along with a brand persona, favorite item, purchasing behavior insights, and a witty email headline:

**Cluster 1: The "Weekend Warrior"**

*   **Data Summary:** Moderate spending ($45.62), low order frequency (1.33 orders), and a long time since last order (283.28 days).
*   **Brand Persona:** *The Occasional Enthusiast* - This person enjoys sports but isn't a hardcore athlete. They might play a sport recreationally on weekends, or simply like to look the part. They're value-conscious and likely to be influenced by sales and promotions.
*   **Favorite Retail Item:** A comfortable, good-looking *sports-inspired t-shirt or a basic pair of athletic shorts*. Something versatile and affordable.
*   **Purchasing Behavior:** They buy items when they need to replace something or when a good deal catches their eye. They aren't brand

**Bonus inspiriation!**
If you'd like to use the Gemini API directly in BigQuery, see this notebook [Using Vertex AI LLMs with data in BigQuery](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/use-cases/applying-llms-to-data/bigquery_ml_llm.ipynb) - and learn how to use the BigQuery function [`ML.GENERATE_TEXT`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-generate-text). Using `ML_GENERATE_TEXT` you can combine cluster analysis and explainations all within BigQuery.

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.
